In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from model import CNNet as CNNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
in_channel = 1
out_channel = 10
batch_size = 64
learning_rate = 0.001
epochs = 3

In [15]:
train_dataset = datasets.MNIST(root='../downloads/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='../downloads/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [16]:
model = CNNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")

In [18]:
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)

        # forward
        scores = model(data)
        loss = criterion(scores, target)

        # backward
        optimizer.zero_grad()
        loss.backward()

        #optimizer step
        optimizer.step()

    print(f"Epoch {epoch+1} / {epochs}")
    check_accuracy(test_loader, model)

Epoch 1 / 3
Checking accuracy on test data
Got 9504 / 10000 with accuracy 95.04
Epoch 2 / 3
Checking accuracy on test data
Got 9695 / 10000 with accuracy 96.95
Epoch 3 / 3
Checking accuracy on test data
Got 9758 / 10000 with accuracy 97.58
